<a href="https://colab.research.google.com/github/fatemafaria142/Evaluating-LLMs-Generated-Food-Recipes-in-Comparison-to-Established-Reference-Recipes/blob/main/Recipe_Generation_using_Llama_2_7b_chat_hf_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **Dataset Link:** https://huggingface.co/datasets/recipe_nlg

In [10]:
import pandas as pd

# Load the original dataset
dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/full_dataset.csv")
dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [14]:
from datasets import Dataset, DatasetDict

# Create a DatasetDict for instruct_tune_dataset
instruct_tune_dataset = DatasetDict()

# Select 1500 rows for the 'train' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["train"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].head(800))

# Select 200 rows for the 'test' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["test"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].tail(200))

# Print the updated instruct_tune_dataset
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 800
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})


#### **Creating a Mental Health Support Prompt Using LangChain**

In [23]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Creating prompt for large language model
    pre_prompt = """[INST]<<SYS>>\nImagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:\n"""
    prompt = pre_prompt + "{ingredients}" +"[\INST]"+"\n{directions}"

    # Extract ingredients and directions from the sample
    example_ingredients = sample['ingredients']
    example_directions = sample['directions']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["ingredients", "directions"])
    single_turn_prompt = prompt_template.format(ingredients=example_ingredients, directions=example_directions)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [24]:
prompt = create_prompt(instruct_tune_dataset["train"][0])
print(prompt)

[INST]<<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"][\INST]
["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]


In [25]:
prompt = create_prompt(instruct_tune_dataset["train"][1])
print(prompt)

[INST]<<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"][\INST]
["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\u00b0 for 3 hours."]


In [26]:
prompt = create_prompt(instruct_tune_dataset["train"][2])
print(prompt)

[INST]<<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"][\INST]
["In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings."]


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [27]:
prompt = create_prompt(instruct_tune_dataset["test"][0])
print(prompt)

[INST]<<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
["1/2 inch piece ginger, minced", "1 1/2 ounces sweet vermouth", "Ice", "1 1/2 ounces gin", "1 1/2 ounces Campari", "Orange slices"][\INST]
["Add the ginger, 1/2 ounce sweet vermouth and ice to a cocktail shaker and muddle.", "Add more ice, more sweet vermouth, gin and the Campari.", "Shake to combine.", "Pour over an ice filled rocks glass.", "Top off with water and garnish with orange slices."]


In [28]:
prompt = create_prompt(instruct_tune_dataset["test"][1])
print(prompt)

[INST]<<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
["1/2 c. orange juice", "1/2 c. raisins", "1/4 c. soy sauce", "1/4 c. white vinegar", "2 Tbsp. Dijon mustard", "1 Tbsp. grated orange peel", "2 Tbsp. ketchup", "2 Tbsp. chili sauce"][\INST]
["1.", "Bring to boil 2 min stirring constantly.", "2.", "Remove from heat.", "Allow to cold to lukewarm.", "3.", "Pour mix into blender till it is pureed.", "4.", "Pour into bottle and cap tightly.", "chill, and use within 90 days."]


In [29]:
prompt = create_prompt(instruct_tune_dataset["test"][2])
print(prompt)

[INST]<<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
["15 lbs boneless ham, fully cooked", "34 cup davinci sugar-free maple syrup", "34 cup Dijon mustard", "1 teaspoon cinnamon"][\INST]
["Soak about 4 hickory wood chunks in water and drain.", "Clean and trim.", "excess fat from ham.", "Preheat grill and prepare for indirect cooking.", "with a large drip pan under where the ham will go.", "Pour in hot tap.", "water to fill pan half full.", "Add soaked wood chips to the fire.", "Oil.", "hot grid to help prevent sticking.", "Place ham on grid directly above.", "drip pan.", "Grill ham, on a covered grill 20-30 minutes per pound, until meat.", "thermometer in thickest part registers 140 degrees F. For best.", "flavor, cook slowly over low coals, adding a few briquettes to both.", "sides of the fire every hour, or as necessary, to m

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [31]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [32]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [33]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [34]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [39]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.7  # You can adjust this value based on your preference
temperature = 0.9  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id,  top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)

# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nImagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:\n"""
prompt = pre_prompt + "{ingredients}[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["ingredients"])

chain = prompt | llm

ingredients = "500g TOBLERONE* Milk Chocolate, chopped", "1 tablespoon instant coffee, dissolved in 2 tablespoons boiling water", "150g butter, chopped", "1 cup chopped raisins", "1 teaspoon vanilla essence"

print(chain.invoke({"ingredients": ingredients}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Great! Let's get started. Here's a recipe for a delicious and unique chocolate dessert that combines the richness of toffee with the sweetness of raisins and the depth of coffee:

Toffee Raisin Chocolate Muffins

Ingredients:

* 500g TOBLERONE* Milk Chocolate, chopped
* 1 tablespoon instant coffee, dissolved in 2 tablespoons boiling water
* 150g butter, chopped
* 1 cup chopped raisins
* 1 teaspoon vanilla essence

Instructions:

1. Preheat the oven to 180°C (350°F). Line a 12-cup muffin tin with muffin liners.
2. In a large mixing bowl, combine the chopped chocolate, coffee, and butter. Stir until the butter has melted and the mixture is smooth.
3. Add the chopped raisins and vanilla essence to the bowl and stir until they are evenly distributed.
4. Pour the mixture into the prepared muffin tin, filling each cup about 3/4 of the way full.
5. Bake for 18-20 minutes, or until the muffins are golden brown and a toothpick inserted into the center comes out clean.
6. Remove the muffins fr

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [40]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [41]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [42]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [43]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],      # Pass train_subset as train_dataset
  eval_dataset=instruct_tune_dataset["test"]         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [44]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.730800
20,1.323000
30,1.127300
40,1.016700
50,0.957600
60,0.945900
70,0.886100
80,0.898100
90,0.877800
100,0.831900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=150, training_loss=0.9925990358988444, metrics={'train_runtime': 1085.1261, 'train_samples_per_second': 0.553, 'train_steps_per_second': 0.138, 'total_flos': 1.2186386694144e+16, 'train_loss': 0.9925990358988444, 'epoch': 1.66})

### **Save the model**

In [45]:
trainer.save_model("Llama_instruct_generation")

In [46]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

# **Conversation Buffer memory**

In [47]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nImagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:\n"""
prompt = pre_prompt + "{ingredients}[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["ingredients"])

In [49]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [53]:
ingredients = "24 ounces lean ground beef", "5 tomatoes, diced", "6 cloves garlic, chopped", "2 tablespoons chili powder", "1 tablespoon cumin", "16 ounces low-sodium refried beans", "7 ounces diced green chilies", "8 fluid ounces no-salt-added tomato sauce", "12 flour tortillas", "12 cup shredded monterey jack cheese", "1 head lettuce, shredded", "2 avocados, pitted,peeled and mashed", "8 fluid ounces sour cream"
chat_llm_chain.predict(ingredients=ingredients)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
('24 ounces lean ground beef', '5 tomatoes, diced', '6 cloves garlic, chopped', '2 tablespoons chili powder', '1 tablespoon cumin', '16 ounces low-sodium refried beans', '7 ounces diced green chilies', '8 fluid ounces no-salt-added tomato sauce', '12 flour tortillas', '12 cup shredded monterey jack cheese', '1 head lettuce, shredded', '2 avocados, pitted,peeled and mashed', '8 fluid ounces sour cream')[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Great! Let\'s start with the dish name. How about "Spicy Beef and Bean Burritos"? Sounds delicious, right? Now, let\'s move on to the instructions.\n\nStep 1: Preheat your oven to 350 degrees Fahrenheit. While you\'re waiting for it to heat up, let\'s start cooking the ground beef. In a large skillet, brown the beef over medium-high heat, breaking it up into small pieces as it cooks. Once it\'s browned, drain the excess fat and add the chili powder, cumin, and garlic. Cook for an additional 2-3 minutes, stirring constantly, until the spices are fragrant and the beef is fully coated.\n\nStep 2: In a separate pan, heat the refried beans over medium heat. Add the diced green chilies and stir to combine. Cook for an additional 2-3 minutes, until the beans are heated through and the chilies are tender.\n\nStep 3: In a third pan, heat the tomato sauce over medium heat. Add the shredded cheese and stir until it\'s melted and combined with the sauce.\n\nStep 4: Now it\'s time to assemble th

#### **Example: 02**

In [54]:
ingredients = "3 medium tomatoes seeded, chopped (about 3 cups)", "1/2 cup scallions, spring or green onions sliced", "1/2 cup green bell peppers chopped", "3 tablespoons lime juice", "2 tablespoons cilantro fresh snipped", "1 tablespoon jalapeno pepper finely chopped", "1 clove garlic up to 3 cloves", "1/2 teaspoons salt"
chat_llm_chain.predict(ingredients=ingredients)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
('3 medium tomatoes seeded, chopped (about 3 cups)', '1/2 cup scallions, spring or green onions sliced', '1/2 cup green bell peppers chopped', '3 tablespoons lime juice', '2 tablespoons cilantro fresh snipped', '1 tablespoon jalapeno pepper finely chopped', '1 clove garlic up to 3 cloves', '1/2 teaspoons salt')[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Great! Let's get started on this delicious dish. Here are the steps to create a mouthwatering Tomato and Green Pepper Salsa:\n\nStep 1: Chop the Tomatoes\n\n* Chop the 3 medium tomatoes seeded and chopped into small pieces. This will help them release their juices and create a smoother texture in the salsa.\n\nStep 2: Prepare the Scallions\n\n* Chop the 1/2 cup of scallions into thin slices. This will give the salsa a fresh and crunchy texture.\n\nStep 3: Chop the Green Peppers\n\n* Chop the 1/2 cup of green bell peppers into small pieces. This will help release their flavor and texture in the salsa.\n\nStep 4: Add the Lime Juice\n\n* Add the 3 tablespoons of lime juice to the chopped tomatoes, scallions, and green peppers. This will help to balance the flavors and give the salsa a tangy taste.\n\nStep 5: Add the Cilantro and Jalapeno\n\n* Add the 2 tablespoons of fresh cilantro snipped and the 1 tablespoon of jalapeno pepper finely chopped to the mixture. This will give the salsa a

#### **Example: 03**

In [55]:
ingredients = "1 (10 ounce) package frozen raspberries, thawed", "2 (3 ounce) packages raspberry Jell-O gelatin", "2 cups boiling water", "1 pint vanilla ice cream", "1 (6 ounce) can frozen lemonade, thawed", "12 cup chopped pecans"
chat_llm_chain.predict(ingredients=ingredients)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients:
('1 (10 ounce) package frozen raspberries, thawed', '2 (3 ounce) packages raspberry Jell-O gelatin', '2 cups boiling water', '1 pint vanilla ice cream', '1 (6 ounce) can frozen lemonade, thawed', '12 cup chopped pecans')[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Great! Let's get started. Here's a recipe for a delicious Raspberry Lemonade Pecan Trifle that combines the freshness of raspberries with the tanginess of lemonade and the crunch of pecans.\n\nIngredients:\n\n* 1 (10 ounce) package frozen raspberries, thawed\n* 2 (3 ounce) packages raspberry Jell-O gelatin\n* 2 cups boiling water\n* 1 pint vanilla ice cream\n* 1 (6 ounce) can frozen lemonade, thawed\n* 12 cup chopped pecans\n\nInstructions:\n\n1. Preheat your oven to 350°F (180°C).\n2. In a small bowl, sprinkle the gelatin over 1/2 cup of the boiling water and let it sit for 5 minutes to soften.\n3. In a separate bowl, mix the raspberries and lemonade until well combined.\n4. In a 9x13 inch baking dish, layer the softened gelatin, raspberry mixture, and chopped pecans in the following order: gelatin, raspberries, pecans. Repeat this process two more times, ending with a layer of pecans on top.\n5. Pour the vanilla ice cream over the layers, making sure to cover everything evenly.\n6

In [56]:
memory.load_memory_variables({})

{'history': 'Human: [\'24 ounces lean ground beef\', \'5 tomatoes, diced\', \'6 cloves garlic, chopped\', \'2 tablespoons chili powder\', \'1 tablespoon cumin\', \'16 ounces low-sodium refried beans\', \'7 ounces diced green chilies\', \'8 fluid ounces no-salt-added tomato sauce\', \'12 flour tortillas\', \'12 cup shredded monterey jack cheese\', \'1 head lettuce, shredded\', \'2 avocados, pitted,peeled and mashed\', \'8 fluid ounces sour cream\']\nAI:   Great! Let\'s start with the dish name. How about "Spicy Beef and Bean Burritos"? Sounds delicious, right? Now, let\'s move on to the instructions.\n\nStep 1: Preheat your oven to 350 degrees Fahrenheit. While you\'re waiting for it to heat up, let\'s start cooking the ground beef. In a large skillet, brown the beef over medium-high heat, breaking it up into small pieces as it cooks. Once it\'s browned, drain the excess fat and add the chili powder, cumin, and garlic. Cook for an additional 2-3 minutes, stirring constantly, until the s

In [57]:
print(memory.buffer)

Human: ['24 ounces lean ground beef', '5 tomatoes, diced', '6 cloves garlic, chopped', '2 tablespoons chili powder', '1 tablespoon cumin', '16 ounces low-sodium refried beans', '7 ounces diced green chilies', '8 fluid ounces no-salt-added tomato sauce', '12 flour tortillas', '12 cup shredded monterey jack cheese', '1 head lettuce, shredded', '2 avocados, pitted,peeled and mashed', '8 fluid ounces sour cream']
AI:   Great! Let's start with the dish name. How about "Spicy Beef and Bean Burritos"? Sounds delicious, right? Now, let's move on to the instructions.

Step 1: Preheat your oven to 350 degrees Fahrenheit. While you're waiting for it to heat up, let's start cooking the ground beef. In a large skillet, brown the beef over medium-high heat, breaking it up into small pieces as it cooks. Once it's browned, drain the excess fat and add the chili powder, cumin, and garlic. Cook for an additional 2-3 minutes, stirring constantly, until the spices are fragrant and the beef is fully coated